<a href="https://colab.research.google.com/github/pfmoro/VisualizacaoRefugiadosVenezuela/blob/main/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px
import os
from scipy.stats import spearmanr
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
from plotly.subplots import make_subplots

In [2]:
# Suponha que você queira remover todos os arquivos CSV antigos
# Lista os arquivos atuais no ambiente de trabalho do Colab
files_in_colab = os.listdir('/content')

for file_name in files_in_colab:
    if file_name.endswith('.csv'):
        os.remove(file_name)
        print(f'Arquivo {file_name} removido.')

In [3]:
uploaded = files.upload()

Saving Oil Refugees GDP.csv to Oil Refugees GDP.csv


In [4]:
# Suponha que o nome do arquivo seja 'dados.csv'
# Se o nome do seu arquivo for diferente, substitua 'dados.csv' pelo nome correto
file_name = 'Oil Refugees GDP.csv'

# Verifica se o arquivo foi carregado corretamente
if file_name in uploaded:
    # Lê o arquivo CSV usando a biblioteca Pandas
    df_oil_ref_gdp = pd.read_csv(file_name,sep=";",encoding='latin-1')
else:
    print(f"O arquivo '{file_name}' não foi encontrado.")

In [5]:
df_oil_ref_gdp = df_oil_ref_gdp.replace('#DIV/0!', pd.NaT if 'datetime' in df_oil_ref_gdp.columns else pd.NA)

In [6]:
df_oil_ref_gdp.head(5)

,Year,Refugees Venezuela,Refugees Venezuela Variation,refugees normalized,Refugees Venezuela Variation %,Crude Oil Year Average,Crude Oil Year Average Variation %,Crude Oil Year Average Variation % Lag +,Crude Oil Year Average Variation % Lag 2+,Crude Oil Year Average Variation % Lag 3+,Crude Oil Year Average Variation % Lag -,GDP,GDP Variation
0,2001.0,798.0,NaN,NaN,NaN,24.351825,-0.137,NaN,NaN,NaN,0.0237,300198.0,NaN
1,2002.0,1304.0,506.0,0.000168,0.63,24.927748,0.024,-0.1374,NaN,NaN,0.1593,277865.0,-0.0744
2,2003.0,2614.0,1310.0,0.000435,1.00,28.898903,0.159,0.0237,-0.1374,NaN,0.3057,261376.0,-0.0593
3,2004.0,3933.0,1319.0,0.000438,0.50,37.733388,0.306,0.1593,0.0237,-0.1374,0.4150,317486.0,0.2147
4,2005.0,5397.0,1464.0,0.000486,0.37,53.391025,0.415,0.3057,0.1593,0.0237,0.2041,361227.0,0.1378


Gráfico simples: PIB x Ano

In [7]:
fig = go.Figure([go.Scatter(x=df_oil_ref_gdp['Year'], y=df_oil_ref_gdp['GDP'])])
fig.show()

Duas variáveis

In [8]:
df_oil_ref_gdp.columns

Index(['Year', 'Refugees Venezuela', 'Refugees Venezuela Variation ',
       'refugees normalized', 'Refugees Venezuela Variation %',
       'Crude Oil Year Average', 'Crude Oil Year Average Variation %',
       'Crude Oil Year Average Variation % Lag +',
       'Crude Oil Year Average Variation % Lag 2+',
       'Crude Oil Year Average Variation % Lag 3+',
       'Crude Oil Year Average Variation % Lag -', 'GDP', 'GDP Variation'],
      dtype='object')

In [9]:
#Remove NaN para plot:
df_plot=df_oil_ref_gdp[['Year','GDP Variation','Crude Oil Year Average Variation %']].dropna(how='any')

# Cria um gráfico de dispersão para a variável A
trace1 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['GDP Variation'].astype(float),
    mode='lines+markers',
    name='Variação do PIB'
)

# Cria um gráfico de dispersão para a variável B
trace2 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['Crude Oil Year Average Variation %'].astype(float),
    mode='lines+markers',
    name='Variação do Preço do Pétroleo'
)

# Define os dados que serão plotados
data = [trace1, trace2]

# Define o layout do gráfico
layout = go.Layout(
    title='Comparação entre Variação do PIB e Preço do Petróleo ao longo dos anos',
    xaxis=dict(title='Ano'),
    yaxis=dict(title='Valor'),
    showlegend=True
)

# Cria a figura com os dados e o layout
fig = go.Figure(data=data, layout=layout)

# Plota o gráfico no Google Colab
#plot(fig)
fig.show()


In [10]:
def rolling_spearman_slow(seqa, seqb, window):
    stridea = seqa.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    corrs = [spearmanr(a, b)[0] for (a,b) in zip(ssa, ssb)]
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)


In [20]:
#Remove NaN para plot:
df_plot=df_oil_ref_gdp[['Year','GDP Variation','Crude Oil Year Average Variation %']].dropna(how='any')
df_plot['GDP Variation']=df_plot['GDP Variation'].astype(float)
df_plot['Crude Oil Year Average Variation %']=df_plot['Crude Oil Year Average Variation %'].astype(float)
#Criar correlação Rolante:
df_plot['correlacao_rolante']=rolling_spearman_slow(df_plot['GDP Variation'].astype(float).values,df_plot['Crude Oil Year Average Variation %'].astype(float).values,5)
df_plot['residuos']=df_plot['GDP Variation']-df_plot['Crude Oil Year Average Variation %']

#Usaremos subplots:
fig = make_subplots(rows=3, cols=1)

# Cria um gráfico de dispersão para a variável A
trace1 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['GDP Variation'].astype(float),
    mode='lines+markers',
    name='Variação do PIB'

)

# Cria um gráfico de dispersão para a variável B
trace2 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['Crude Oil Year Average Variation %'].astype(float),
    mode='lines+markers',
    name='Variação do Preço do Pétroleo'
)

# Gráfico de correlação (usaremos um scatter plot simples para demonstrar)
trace3 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['correlacao_rolante'],
    mode='lines',
    name='Correlação Rolante (Spearman)'
)

# Gráfico de resíduos
trace4 = go.Scatter(
    x=df_plot['Year'],
    y=df_plot['residuos'],
    mode='lines+markers',
    name='Resíduos'
)

# Encontrar os índices onde a correlação em módulo é maior que 0.7
indices_correlacao_alta = np.where(np.abs(df_plot['correlacao_rolante']) > 0.7)[0]

# Inicializar uma lista para armazenar os intervalos de início e fim
intervalos = []
# Encontrar os subintervalos consecutivos
for i in range(len(indices_correlacao_alta)):
    if i == 0 or indices_correlacao_alta[i] != indices_correlacao_alta[i-1] + 1:
        # Início de um novo intervalo
        inicio = indices_correlacao_alta[i]
    if i == len(indices_correlacao_alta) - 1 or indices_correlacao_alta[i] != indices_correlacao_alta[i+1] - 1:
        # Fim de um intervalo
        fim = indices_correlacao_alta[i]
        intervalos.append((inicio, fim))

# Define os dados que serão plotados
data = [trace1, trace2, trace3, trace4]
fig.add_trace(trace1,row=1, col=1)
fig.add_trace(trace2,row=1, col=1)
fig.add_trace(trace3,row=2, col=1)
fig.add_trace(trace4,row=3, col=1)

fig.update_layout(height=600, width=800,
                  title_text="Comparação entre Variável A e Variável B, Correlação e Resíduos",
                  shapes=[
        # Adicionar uma forma para cada intervalo de alta correlação
        dict(
            type='rect',
            x0=df_plot['Year'].iloc[inicio],
            y0=-1, #min(df_plot['GDP Variation']),
            x1=df_plot['Year'].iloc[fim],
            y1=1, #max(df_plot['GDP Variation']),
            fillcolor='Lightgrey',
            opacity=0.5,
            line=dict(
                width=0,
            ),
        )
        for inicio, fim in intervalos
    ]
                  )


# Plota o gráfico no Google Colab
#plot(fig)
fig.show()

